# 根据要求导出`WRF-CMAQ`模拟数据
2、	四个国控点的数据。各指标如O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献和敏感性指标（HNO3/H2O2比值）的时间序列，可以存成一个excel表格，例如第一列是时间（小时），第二列是O3浓度，第三列….。

---
*@author: Evan*\
*@date: 2023-05-15*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../src/')
from findpoint import findpoint
import ModelEvaLib as me

In [2]:
chemFile = xr.open_dataset('F:/Data/case_anqing/April/April_chem.nc')
metFile  = xr.open_dataset('F:/Data/case_anqing/April/April_met.nc')
isam = xr.open_dataset('F:/Data/case_anqing/April/April_ISAM1.nc')
pa=xr.open_dataset('F:/Data/case_anqing/April/April_PA1.nc')

lat = chemFile.latitude
lon = chemFile.longitude

ncfile = chemFile.O3

nlevel = 21
O3   = chemFile.O3[:,:nlevel,:,:]
NO2  = chemFile.NO2[:,:nlevel,:,:]
PM25 = chemFile.PM25[:,:nlevel,:,:]
ht   = chemFile.HT[:,:nlevel,:,:]

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

AQ  = isam.O3_AQ[:,:nlevel,:,:]
HF  = isam.O3_HF[:,:nlevel,:,:]
TL  = isam.O3_TL[:,:nlevel,:,:]
CZ  = isam.O3_CZ[:,:nlevel,:,:]
JJ  = isam.O3_JJ[:,:nlevel,:,:]
OTH = isam.O3_OTH[:,:nlevel,:,:]
BCO = isam.O3_BCO[:,:nlevel,:,:]
ICO = isam.O3_ICO[:,:nlevel,:,:]

HNO3 = pa.HNO3prod[:,:nlevel,:,:]
H2O2 = pa.H2O2prod[:,:nlevel,:,:]

In [3]:
siteloc = pd.read_excel('F:/Data/case_anqing/April/obsdata/安庆市站点坐标.xlsx',header=0,index_col=0)

sites = ['安庆大学','市人大','环科院','马山宾馆','安庆政务中心']

lono = siteloc[siteloc.index.isin(sites)]['经度']
lato = siteloc[siteloc.index.isin(sites)]['纬度']

lono

站点名称
安庆大学      116.9898
市人大       117.0597
环科院       117.0807
马山宾馆      117.0331
安庆政务中心    117.1150
Name: 经度, dtype: float64

In [3]:
# 三个市区站点的中心点位
test = chemFile.O3[:,:,56,74]

print(test.longitude.values)
print(test.latitude.values)

117.05347
30.522572


In [6]:
x_index=[]
y_index=[]


out_x, out_y = findpoint(117.05347,30.522572,ncfile)
x_index.append(out_x)
y_index.append(out_y)

O3_sel   = O3.isel(x=x_index, y=y_index).squeeze()
NO2_sel  = NO2.isel(x=x_index, y=y_index).squeeze()
PM25_sel = PM25.isel(x=x_index, y=y_index).squeeze()
htsel    = ht.isel(x=x_index, y=y_index).squeeze()

HADV_sel = HADV.isel(x=x_index, y=y_index).squeeze()
ZADV_sel = ZADV.isel(x=x_index, y=y_index).squeeze()
HDIF_sel = HDIF.isel(x=x_index, y=y_index).squeeze()
VDIF_sel = VDIF.isel(x=x_index, y=y_index).squeeze()
DDEP_sel = DDEP.isel(x=x_index, y=y_index).squeeze()
CHEM_sel = CHEM.isel(x=x_index, y=y_index).squeeze()

HNO3_sel = HNO3.isel(x=x_index, y=y_index).squeeze()
H2O2_sel = H2O2.isel(x=x_index, y=y_index).squeeze()

AQ_sel  = AQ.isel(x=x_index, y=y_index).squeeze()
HF_sel  = HF.isel(x=x_index, y=y_index).squeeze()
TL_sel  = TL.isel(x=x_index, y=y_index).squeeze()
CZ_sel  = CZ.isel(x=x_index, y=y_index).squeeze()
JJ_sel  = JJ.isel(x=x_index, y=y_index).squeeze()
OTH_sel = OTH.isel(x=x_index, y=y_index).squeeze()
BCO_sel = BCO.isel(x=x_index, y=y_index).squeeze()
ICO_sel = ICO.isel(x=x_index, y=y_index).squeeze()

In [8]:
savemat(
    'D:/Download/centralpoint.mat',
    {
    'height': htsel,
    'O3'    : O3_sel,
    'NO2'   : NO2_sel,
    'PM25'  : PM25_sel,
    'HADV'  : HADV_sel,
    'ZADV'  : ZADV_sel,
    'HDIF'  : HDIF_sel,
    'VDIF'  : VDIF_sel,
    'DDEP'  : DDEP_sel,
    'CHEM'  : CHEM_sel,
    'AQ'    : AQ_sel,
    'HF'    : HF_sel,
    'TL'    : TL_sel,
    'CZ'    : CZ_sel,
    'JJ'    : JJ_sel,
    'OTH'   : OTH_sel,
    'BCO'   : BCO_sel,
    'ICO'   : ICO_sel
    }
    )